## Imports

In [45]:
import pandas as pd
import sklearn

ModuleNotFoundError: No module named 'sklearn'

## Read data

In [35]:
def add_zero_padding(number):
    if len(str(number)) == 1: return "0" + str(number)
    else: return number

In [40]:
dfYellow = pd.read_csv("taxi_data\yellow_tripdata_2019_concat.csv", chunksize=1000000)

